In [ ]:
#IMPORTANT: USE ONLY IF NECCESARY

from sklearn.model_selection import train_test_split

# Assuming X_train and y_train are your original data
# Set the test_size and validation_size according to your needs
test_size = 0.2  # 80% training, 20% test
validation_size = 0.2  # 80% training, 20% validation
random_seed = 42  # for reproducibility

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=test_size, random_state=random_seed)

# Split training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_size, random_state=random_seed)

In [ ]:
def normalize(X):
    v_min = X.min(axis=(2, 3), keepdims=True)
    v_max = X.max(axis=(2, 3), keepdims=True)

    # Calculate the range and replace zero values with epsilon to avoid division by zero
    range_values = np.where((v_max - v_min) == 0, np.finfo(float).eps, v_max - v_min)

    # Normalize the data
    X_normalized = np.where(range_values == 0, 0, (X - v_min) / range_values)

    return X_normalized

In [ ]:
from keras.utils import to_categorical
X_train = normalize(X_train)
X_val = normalize(X_val)
X_test = normalize(X_test)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
y_val = to_categorical(y_val, 10)

from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)
X_val, y_val = shuffle(X_val, y_val, random_state=0)

In [ ]:
#ONLY IF NECCESARY
X_train = np.expand_dims(X_train, axis=4)
X_val = np.expand_dims(X_val, axis=4)
X_test = np.expand_dims(X_test, axis=4)

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Activation, ZeroPadding3D, TimeDistributed, LSTM, GRU, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv3D, MaxPooling3D

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


model_adam_lstm = Sequential()

# Conv3D layers
model_adam_lstm.add(Conv3D(64, (3, 3, 3), strides=1, input_shape=(22, 100, 100, 3), activation='relu', padding='valid'))
model_adam_lstm.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model_adam_lstm.add(Conv3D(128, (3, 3, 3), activation='relu', strides=1))
model_adam_lstm.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model_adam_lstm.add(Conv3D(256, (2, 2, 2), activation='relu', strides=1))
model_adam_lstm.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model_adam_lstm.add(Conv3D(512, (1, 1, 1), activation='relu', strides=1))
model_adam_lstm.add(MaxPooling3D(pool_size=(1, 1, 1), strides=2))

# Reshape output before LSTM layer
model_adam_lstm.add(Reshape((-1, 512)))  # Reshape to (timesteps, features)

# LSTM layer
model_adam_lstm.add(LSTM(50))  # You can adjust the number of LSTM units as needed

# FC layers group
model_adam_lstm.add(Dense(4096, activation='relu'))
model_adam_lstm.add(Dropout(0.5))
model_adam_lstm.add(Dense(2048, activation='relu'))
model_adam_lstm.add(Dropout(0.5))
model_adam_lstm.add(Dense(1024, activation='relu'))
model_adam_lstm.add(Dropout(0.5))
model_adam_lstm.add(Dense(10, activation='softmax'))

model_adam_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




In [ ]:
epochs=200
model_adam_lstm = model_adam_lstm.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))
plt.figure(figsize=(16, 8))

plt.plot(model_adam_lstm.history['accuracy'], label='Training Accuracy')
plt.plot(model_adam_lstm.history['val_accuracy'], label='Validation Accuracy')
plt.title('CNN with LSTM')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
